# 🩻 X-Ray Chest - Interactive Training Console

## 🛑 PREREQUISITE: Absolute Path Configuration
This notebook uses an **explicit path** strategy. You must copy-paste the REAL absolute paths from your OS below.
Do not assume the code knows where your files are.

In [ ]:
import os
import sys
import torch
import warnings

# ==========================================
# 👇 USER CONFIGURATION (EDIT THIS CELL!) 👇
# ==========================================

# 1. Where is the 'src' folder containing the python code?
SRC_PATH = r"c:\Users\albaa\Desktop\X-Ray-Chest\src"  

# 2. Where is the CSV file?
CSV_PATH = r"c:\Users\albaa\Desktop\X-Ray-Chest\data\Data_Entry_2017.csv"

# 3. Where is the folder containing images?
IMAGES_PATH = r"c:\Users\albaa\Desktop\X-Ray-Chest\images-224"

# ==========================================

# Check paths immediately
for p in [SRC_PATH, CSV_PATH, IMAGES_PATH]:
    if not os.path.exists(p):
        raise FileNotFoundError(f"❌ ERROR: Path not found: {p}")

print("✅ All paths verified.")

# Add source code to Python Path to allow imports
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

import main
from dataset import Dataset
from dataloader import create_dataloader
from model import MultiLabelResNet

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Accelerating on: {device}")

## 🚀 Training Pipeline

In [ ]:
# Hyperparameters
BATCH_SIZE = 16
EPOCHS = 5
LR = 1e-4
IMG_SIZE = 224
NUM_WORKERS = 0 # Keep 0 for Windows

# --- 1. DATA LOADING ---
print("\n--- [1] Loading Data ---")
# Note: The Dataset class now strictly enforces the absolute path provided.
dataset_manager = Dataset(CSV_PATH)

# Preprocessing
dataset_manager.clean_column_names()
dataset_manager.clean_labels("label")
dataset_manager.one_hot_encode_labels("label")

# Integrity Check (Will raise error if files are missing)
missing = dataset_manager.check_image_files(IMAGES_PATH)
if missing:
    print(f"⚠️ Found {len(missing)} missing images. Removing them from dataset.")
    dataset_manager.data = dataset_manager.data[~dataset_manager.data['image'].isin(missing)]

dataset_manager.select_relevant_columns()
train_df, val_df, test_df = dataset_manager.patient_split()

print(f"Stats: Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")

# DataLoader
train_loader, val_loader, test_loader = create_dataloader(
    train_df, val_df, test_df,
    IMAGES_PATH,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    image_size=IMG_SIZE
)

In [ ]:
# --- 2. MODEL & OPTIMIZER ---
import torch.optim as optim
import torch.nn as nn

labels_list = [c for c in train_df.columns if c not in ['image', 'patientid']]
print(f"\n--- [2] Model Setup ({len(labels_list)} Classes) ---")

model = MultiLabelResNet(num_classes=len(labels_list)).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [ ]:
# --- 3. TRAINING LOOP ---
from src.main import train_one_epoch, validate # Re-using logic from main.py if possible, or define here.
# Since main.py functions are stand-alone, we can import them!

print("\n--- [3] Start Training ---")

best_auc = 0.0

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    
    # Train
    t_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    v_loss, v_auc, _ = validate(model, val_loader, criterion, device, labels_list)
    
    print(f" -> Train Loss: {t_loss:.4f} | Val Loss: {v_loss:.4f} | Val Mean AUC: {v_auc:.4f}")
    
    if v_auc > best_auc:
        best_auc = v_auc
        torch.save(model.state_dict(), "best_model_notebook.pth")
        print("⚡ Best Model Saved")

print("DONE.")